1. Files and Packages

In [ ]:
import os, time, glob, shutil, subprocess as sp
from pathlib import Path
from datetime import datetime
import pandas as pd
import numpy as np
import yaml
from jinja2 import Environment, FileSystemLoader
import optuna

In [ ]:
# === Files that have to be adapted ===
HOME = Path("/home/lfuchs")
SCRIPTS = HOME / "masterarbeit" / "scripts"
CONFIG  = HOME / "masterarbeit" / "config"

REQUEST_TPL_DIR = SCRIPTS / "request_templates"
REQUEST_TPL_NAME = "backtest_template.request.j2" 

REQUEST_OUT_DIR  = HOME / "simulation-requests"
RESULTS_BASE     = HOME / "simulation-results"

GREPPER = Path("/home/mgalas/git/FWTClientRepo/src/main/python/fwt/tearsheets/Grepper.py")

2. Bayesian Optimisation Algorithm 

In [ ]:
# --- Helper function to load the search space --- 
# Note: the search space can be easily modified by changing the search_space.yaml template
def load_search_space(path=CONFIG/"search_space.yaml"):
    with open(path, "r") as f:
        return yaml.safe_load(f)

In [ ]:
# --- Helper function to create a request file ---
# Note: the template only needs to be changed if DRACUS requires a new/updated request file format
def generate_request_from_dict(params: dict, out_filename: str):
    env = Environment(loader=FileSystemLoader(str(REQUEST_TPL_DIR)), autoescape=False, trim_blocks=True, lstrip_blocks=True)
    tpl = env.get_template(REQUEST_TPL_NAME)
    text = tpl.render(**params)

    REQUEST_OUT_DIR.mkdir(parents=True, exist_ok=True)
    out_path = REQUEST_OUT_DIR / out_filename
    with open(out_path, "w") as f:
        f.write(text)
    return str(out_path)


In [ ]:
# --- Helper function to generate the ZQQ.csv from the log.zip file ---
def make_zqq_csv(result_dir: str | Path, subject_label: str) -> str:
    result_dir = Path(result_dir)
    deployed = result_dir / "opt" / "simulations-service" / "deployed-jobs" / subject_label

    # 1) Find job-*-log.zip
    zip_files = list(result_dir.glob("job-*-log.zip"))
    if not zip_files:
        raise FileNotFoundError(f"Kein job-*-log.zip in {deployed}")
    
    # 2) Unzip → creates job-*-log.txt
    sp.run(
        ["unzip", "-o", str(zip_files[0])],
        cwd=result_dir,
        check=True,
        stdout=sp.DEVNULL
    )

    # 3) Find job-*-log.txt
    job_logs = sorted(deployed.glob("job-*-log.txt"))
    if not job_logs:
        raise FileNotFoundError(f"Nach unzip kein job-*-log.txt in {deployed} gefunden.")
    jobfile = str(job_logs[-1])

    # 4) Run Grepper
    cmd = ["python3", str(GREPPER), jobfile, subject_label]
    sp.run(cmd, check=True)

    zqq_src = deployed / f"ZQQ_{subject_label}.csv"
    if not zqq_src.exists():
        raise FileNotFoundError(f"{zqq_src} wurde von Grepper.py nicht erzeugt.")

    # 5) Copy ZQQ into the main result directory
    zqq_dst = result_dir / zqq_src.name
    shutil.copy2(zqq_src, zqq_dst)
    return str(zqq_dst)

#Example usage:
#make_zqq_csv("/home/lfuchs/simulation-results/bo_f1_t1", "bo_f1_t1")


In [ ]:
# --- Utility function which calculates utility based on performance metrics in ZQQ.csv ---
# Note: has to be updated if the utility function definition changes
def utility_from_zqq(
    zqq_csv: str | Path,
    fold_start: str, fold_end: str,
    crash_start: str,
    return_components: bool = False,
    lambdas=(0.35, 0.20, 0.15, 0.30)
):
   
    lam1, lam2, lam3, lam4 = lambdas

    df = pd.read_csv(zqq_csv)
    date_col = df.columns[1]
    fund_col = df.columns[2]
    bench_col = df.columns[3]
    df = df.rename(columns={date_col: "Date", fund_col: "Fund", bench_col: "Benchmark"})
    df["Date"] = pd.to_datetime(df["Date"])
    df = df.set_index("Date").sort_index()

    # Restrict to fold period
    fold_start = pd.to_datetime(fold_start)
    fold_end   = pd.to_datetime(fold_end)
    df_fold = df.loc[(df.index >= fold_start) & (df.index <= fold_end)]
    if df_fold.empty:
        return float("-inf")

    # First available day t
    t = df_fold.index[0]

    # Helper: find next available date ≤ target
    def pick_leq(target):
        target = pd.to_datetime(target)
        ix = df_fold.index.searchsorted(target, side="right") - 1
        if ix < 0: ix = 0
        return df_fold.index[ix]

    # Calculate relative returns
    def rel(t1, t2):
        a = df_fold.loc[t2, "Fund"] / df_fold.loc[t1, "Fund"]
        m = df_fold.loc[t2, "Benchmark"] / df_fold.loc[t1, "Benchmark"]
        return float(a), float(m)

    # f1: 5y annualised minus 3.5%
    t1 = pick_leq(t + pd.DateOffset(years=5))
    a1, m1 = rel(t, t1)
    f1 = (a1 ** (1/5)) - (m1 ** (1/5)) - 0.035

    # f2: 1y relative
    t2 = pick_leq(t + pd.DateOffset(years=1))
    a2, m2 = rel(t, t2)
    f2 = a2 - m2

    # f3: 3m relative annualised (^4) + 0.2
    t3 = pick_leq(t + pd.DateOffset(months=3))
    a3, m3 = rel(t, t3)
    f3 = (a3 ** 4) - (m3 ** 4) + 0.2

    # f4: Integral over 2 years starting at crash_start
    crash_start = pick_leq(pd.to_datetime(crash_start))
    crash_end   = pick_leq(crash_start + pd.DateOffset(years=2))
    df_win = df_fold.loc[(df_fold.index >= crash_start) & (df_fold.index <= crash_end)].copy()
    if df_win.empty:
        f4 = 0.0
    else:
        # Normalisation at crash start
        Ac = float(df_fold.loc[crash_start, "Fund"])
        Sc = float(df_fold.loc[crash_start, "Benchmark"])
        fund_norm  = df_win["Fund"] / Ac
        bench_norm = df_win["Benchmark"] / Sc
        df_win["diff"] = np.sqrt(fund_norm) - np.sqrt(bench_norm)
        days = (df_win.index - df_win.index[0]).days.values
        f4 = np.trapz(df_win["diff"].values, x=days) / 2.0

    U = lam1*f1 + lam2*f2 + lam3*f3 + lam4*f4

    if return_components:
        return float(U), float(f1), float(f2), float(f3), float(f4)
    return float(U)

#Example usage:
#u, f1, f2, f3, f4 = utility_from_zqq(RESULTS_BASE / "bo_f1_t0/" "ZQQ_bo_f1_t0.csv", "2005-01-03", "2009-12-31", "2007-07-19", return_components=True)
#print(f"U={u:.4f}, f1={f1:.4f}, f2={f2:.4f}, f3={f3:.4f}, f4={f4:.4f}")

In [ ]:
# --- Helper function to sample hyperparameters from the defined search space ---
# In our case: Generates NPORT, NFREQ and sector-specific Value/Growth weights for use in BO trials
def sample_params_from_space(trial, space, start_date: str, end_date: str):
    
    nport = trial.suggest_int("NPORT", space["nport"]["low"], space["nport"]["high"])
    nfreq = trial.suggest_int("NFREQ", space["nfreq"]["low"], space["nfreq"]["high"])

    sectors = space["sectors"]
    low_w, high_w = space["weight_bounds"]["low"], space["weight_bounds"]["high"]

    VE, GE = {}, {}
    params = {
        "STARTDATE": start_date,
        "ENDDATE": end_date,
        "NPORT": int(nport),
        "NFREQ": int(nfreq),
        "VE": VE,       
        "GE": GE,
    }

    # Important: Add sequentially numbered keys to match {{ VE1 }}, {{ VE2 }} etc. and {{ GE1 }}, {{ GE2 }} etc. in the template
    for i, sec in enumerate(sectors, start=1):
        v = trial.suggest_float(f"VE::{sec}", low_w, high_w)
        g = trial.suggest_float(f"GE::{sec}", low_w, high_w)
        VE[sec] = v
        GE[sec] = g
        params[f"VE{i}"] = v   
        params[f"GE{i}"] = g 

    return params


3. Sequential Bayesian Optimisation Runs

In [ ]:
# --- Definition of cross-validation folds ---
# Note: folds must be adapted; crash dates are required for f4 (crash-resilience component of the utility function); 
# multiple evaluation periods are needed because the utility is always defined on 5-year windows
FOLDS = {
    "f1": {
        "req_start":  "2001-01-02",
        "req_end":    "2009-12-31",
        "eval_start": "2005-01-03",
        "eval_end":   "2009-12-31",
        "crash":      "2007-07-19",
    },
    "f2": {
        "req_start":  "2006-01-03",
        "req_end":    "2014-12-31",
        "eval_start": "2010-01-04",
        "eval_end":   "2014-12-31",
        "crash":      "2011-07-21",
    },
    "f3": {
        "req_start":  "2011-01-03",
        "req_end":    "2019-12-31",
        "eval_start": "2015-01-02",
        "eval_end":   "2019-12-31",
        "crash":      "2018-09-20",
    },
    "f4": {
        "req_start":  "2016-01-04",
        "req_end":    "2024-12-31",
        "eval_start": "2020-01-02",
        "eval_end":   "2024-12-31",
        "crash":      "2020-03-23",
    },
     "f5": {
        "req_start": "2001-01-02",
        "req_end":   "2015-12-31",
        "eval1_start": "2005-01-03",
        "eval1_end":   "2009-12-31",
        "crash1":      "2007-07-19",
        "eval2_start": "2010-01-04",
        "eval2_end":   "2014-12-31",
        "crash2":      "2011-07-21",
    },

    "f6": {
        "req_start": "2001-01-02",
        "req_end":   "2020-12-31",
        "eval1_start": "2005-01-03",
        "eval1_end":   "2009-12-31",
        "crash1":      "2007-07-19",
        "eval2_start": "2010-01-04",
        "eval2_end":   "2014-12-31",
        "crash2":      "2011-07-21",
        "eval3_start": "2015-01-02",
        "eval3_end":   "2019-12-31",
        "crash3":      "2018-09-20",
    },
}


In [ ]:
# --- Objective function for BO runs on Fold 1 ---
# Submits request, waits for DRACUS results, generates ZQQ.csv and computes utility
def objective_real(trial,
                   fold_name: str,
                   req_start: str, req_end: str,      
                   eval_start: str, eval_end: str,          
                   crash_start: str,
                   wait_hours=6, poll_secs=60):
    
    # Generate parameters from the defined search space
    space = load_search_space()
    params = sample_params_from_space(trial, space, req_start, req_end)

    # Create unique label and write request file
    label = f"bo_{fold_name}_t{trial.number}"
    req_file = f"{label}.request"
    generate_request_from_dict(params, req_file)

    # Wait for result directory
    result_dir = RESULTS_BASE / label
    deadline = time.time() + wait_hours*3600
    while time.time() < deadline:
        if result_dir.exists():
            break
        time.sleep(poll_secs)
    if not result_dir.exists():
        return float("-inf")

    # Generate ZQQ.csv
    zqq_path = make_zqq_csv(result_dir, label)

    # Compute utility and store components
    u, f1, f2, f3, f4 = utility_from_zqq(zqq_path, eval_start, eval_end, crash_start, return_components=True)
    trial.set_user_attr("f1", f1)
    trial.set_user_attr("f2", f2)
    trial.set_user_attr("f3", f3)
    trial.set_user_attr("f4", f4)
    
    return float(u)


In [ ]:
# --- Run BO study on fold f1 ---
# Note: adjust n_trials as needed
fold = FOLDS["f1"]
study = optuna.create_study(direction="maximize", study_name="real_f1",
                            storage="sqlite:///real_f1.db", load_if_exists=True)

study.optimize(
    lambda t: objective_real(t, "f1",
                             fold["req_start"], fold["req_end"],
                             fold["eval_start"], fold["eval_end"],
                             fold["crash"]),
    n_trials=4,
    show_progress_bar=True
)

print("Best value:", study.best_value)

In [ ]:
## --- Objective function for Fold 5 ---
# Note: same as other folds, but utility is averaged over two 5-year evaluation periods (2005–2009 + 2010–2014)
def objective_real_f5(trial,
                      fold_name: str,
                      req_start: str, req_end: str,
                      eval1_start: str, eval1_end: str, crash1: str,
                      eval2_start: str, eval2_end: str, crash2: str,
                      wait_hours=6, poll_secs=60):

    space = load_search_space()
    params = sample_params_from_space(trial, space, req_start, req_end)

    label = f"bo_{fold_name}_t{trial.number}"
    req_file = f"{label}.request"
    generate_request_from_dict(params, req_file)

    result_dir = RESULTS_BASE / label
    deadline = time.time() + wait_hours*3600
    while time.time() < deadline:
        if result_dir.exists():
            break
        time.sleep(poll_secs)
    if not result_dir.exists():
        return float("-inf")

    zqq_path = make_zqq_csv(result_dir, label)

    u1, a1, b1, c1, d1 = utility_from_zqq(zqq_path, eval1_start, eval1_end, crash1, return_components=True)
    u2, a2, b2, c2, d2 = utility_from_zqq(zqq_path, eval2_start, eval2_end, crash2, return_components=True)
    u_avg = 0.5 * (u1 + u2)

    trial.set_user_attr("f1", 0.5 * (a1 + a2))
    trial.set_user_attr("f2", 0.5 * (b1 + b2))
    trial.set_user_attr("f3", 0.5 * (c1 + c2))
    trial.set_user_attr("f4", 0.5 * (d1 + d2))

    return float(u_avg)


# --- Run BO study on fold f5 ---
# Note: adjust n_trials as needed
fold5 = FOLDS["f5"]

study5 = optuna.create_study(
    direction="maximize",
    study_name="real_f5",
    storage="sqlite:///real_f5.db",
    load_if_exists=True
)

study5.optimize(
    lambda t: objective_real_f5(
        t, "f5",
        req_start=fold5["req_start"], req_end=fold5["req_end"],
        eval1_start=fold5["eval1_start"], eval1_end=fold5["eval1_end"], crash1=fold5["crash1"],
        eval2_start=fold5["eval2_start"], eval2_end=fold5["eval2_end"], crash2=fold5["crash2"],
    ),
    n_trials=20,
    show_progress_bar=True,
)

print("REAL f5 best value:", study5.best_value)
print("REAL f5 best params:", study5.best_trial.params)


In [ ]:
## --- Objective function for Fold 6 ---
# Note: same as other folds, but utility is averaged over three 5-year evaluation periods (2005–2009 + 2010–2014 + 2015–2019) ---
def objective_real_f6(trial,
                      fold_name: str,
                      req_start: str, req_end: str,
                      eval1_start: str, eval1_end: str, crash1: str,
                      eval2_start: str, eval2_end: str, crash2: str,
                      eval3_start: str, eval3_end: str, crash3: str,
                      wait_hours=6, poll_secs=60):

    space = load_search_space()
    params = sample_params_from_space(trial, space, req_start, req_end)

    label = f"bo_{fold_name}_t{trial.number}"
    req_file = f"{label}.request"
    generate_request_from_dict(params, req_file)

    result_dir = RESULTS_BASE / label
    deadline = time.time() + wait_hours*3600
    while time.time() < deadline:
        if result_dir.exists():
            break
        time.sleep(poll_secs)
    if not result_dir.exists():
        return float("-inf")

    zqq_path = make_zqq_csv(result_dir, label)

    #new
    u1, a1, b1, c1, d1 = utility_from_zqq(zqq_path, eval1_start, eval1_end, crash1, return_components=True)
    u2, a2, b2, c2, d2 = utility_from_zqq(zqq_path, eval2_start, eval2_end, crash2, return_components=True)
    u3, a3, b3, c3, d3 = utility_from_zqq(zqq_path, eval3_start, eval3_end, crash3, return_components=True)
    u_avg = (u1 + u2 + u3) / 3.0

    trial.set_user_attr("f1", (a1 + a2 + a3) / 3.0)
    trial.set_user_attr("f2", (b1 + b2 + b3) / 3.0)
    trial.set_user_attr("f3", (c1 + c2 + c3) / 3.0)
    trial.set_user_attr("f4", (d1 + d2 + d3) / 3.0)

    return float(u_avg)


# --- Run BO study on fold f6 ---
# Note: adjust n_trials as needed
fold6 = FOLDS["f6"]

study6 = optuna.create_study(
    direction="maximize",
    study_name="real_f6",
    storage="sqlite:///real_f6.db",
    load_if_exists=True
)

study6.optimize(
    lambda t: objective_real_f6(
        t, "f6",
        req_start=fold6["req_start"], req_end=fold6["req_end"],
        eval1_start=fold6["eval1_start"], eval1_end=fold6["eval1_end"], crash1=fold6["crash1"],
        eval2_start=fold6["eval2_start"], eval2_end=fold6["eval2_end"], crash2=fold6["crash2"],
        eval3_start=fold6["eval3_start"], eval3_end=fold6["eval3_end"], crash3=fold6["crash3"],
    ),
    n_trials=20,
    show_progress_bar=True
)

print("REAL f6 best value:", study6.best_value)
print("REAL f6 best params:", study6.best_trial.params)



In [ ]:
# --- Load completed BO studies ---
#study = optuna.load_study(study_name="real_f1", storage="sqlite:///real_f1.db")
#df1 = study.trials_dataframe(attrs=("params", "user_attrs", "value", "state", "datetime_start", "datetime_complete"))
#df1.head(25)

#study5 = optuna.load_study(study_name="real_f5", storage="sqlite:///real_f5.db")
#df5 = study5.trials_dataframe(attrs=("params", "user_attrs", "value", "state", "datetime_start", "datetime_complete"))
#df5.head(25)

#study6 = optuna.load_study(study_name="real_f6", storage="sqlite:///real_f6.db")
#df6 = study6.trials_dataframe(attrs=("params", "user_attrs", "value", "state", "datetime_start", "datetime_complete"))
#df6.head(25)

4. Batch (Constant Liar) Bayesian Optimisation Runs

In [ ]:
# --- Send request file and return the label ---
def submit_job_f1(trial, fold, label_prefix="cl_f1"):
    space = load_search_space()
    params = sample_params_from_space(trial, space, fold["req_start"], fold["req_end"])

    label = f"{label_prefix}_t{trial.number}"
    req_file = f"{label}.request"
    generate_request_from_dict(params, req_file)
    return label


# --- Make ZQQ.csv and calculate utility value ---
def score_job_f1(label, fold, wait_hours=8, poll_secs=60):
    result_dir = RESULTS_BASE / label

    deadline = time.time() + wait_hours * 3600
    while time.time() < deadline:
        if result_dir.exists():
            break
        time.sleep(poll_secs)
    if not result_dir.exists():
        return float("-inf"), None, None, None, None, None

    zqq_path = make_zqq_csv(result_dir, label)

    U, f1, f2, f3, f4 = utility_from_zqq(
        zqq_path, fold["eval_start"], fold["eval_end"], fold["crash"],
        return_components=True
    )
    return float(U), float(f1), float(f2), float(f3), float(f4), str(zqq_path)


# --- Batch BO run with Constant Liar on Fold f1 ---
def cl_run_f1(
    batch_size=3,
    n_batches=3,
    lie_policy="best",          
    storage="sqlite:///cl_f1.db",
    study_name="cl_f1",
    seed=42
):
    fold = FOLDS["f1"]

    # Real study (only stores true values)
    study = optuna.create_study(direction="maximize", study_name=study_name,
                                storage=storage, load_if_exists=True,
                                sampler=optuna.samplers.TPESampler(seed=seed))

    # Shadow study (only for lies, in-memory)
    shadow = optuna.create_study(direction=study.direction,
                                 sampler=optuna.samplers.TPESampler(seed=seed))

    # Helper function to determine liar value based on policy
    def liar_value():
        vals = [t.value for t in study.trials if t.value is not None]
        if not vals:
            return 0.0
        if lie_policy == "best":
            return max(vals)
        if lie_policy == "mean":
            return mean(vals)
        if lie_policy == "pessimistic":
            return min(vals)
        return mean(vals)

    for b in range(n_batches):
        trials_real = []
        labels = []

        # Propose a batch via shadow             
        for _ in range(batch_size):
            t_real = study.ask()
            t_shadow = shadow.ask()
            
            space = load_search_space()
            _ = sample_params_from_space(t_real,   space, fold["req_start"], fold["req_end"])
            _ = sample_params_from_space(t_shadow, space, fold["req_start"], fold["req_end"])
            shadow.tell(t_shadow, liar_value()) # immediately lie to the shadow trial

            # Submit request file
            label = submit_job_f1(t_real, fold, label_prefix=f"cl_f1_b{b}")
            trials_real.append(t_real)
            labels.append(label)

        # Score all batch jobs and record true values
        for t_real, label in zip(trials_real, labels):

            U, f1, f2, f3, f4, zqq_path = score_job_f1(label, fold, wait_hours=8, poll_secs=60)
            t_real.set_user_attr("f1", f1)
            t_real.set_user_attr("f2", f2)
            t_real.set_user_attr("f3", f3)
            t_real.set_user_attr("f4", f4)
            t_real.set_user_attr("zqq_path", zqq_path)

            study.tell(t_real, float(U))
            print(f"[CL f1] trial {t_real.number} -> U={U:.4f} (f1={f1:.4f}, f2={f2:.4f}, f3={f3:.4f}, f4={f4:.4f})")

    print("CL f1 best value:", study.best_value)
    print("CL f1 best params:", study.best_trial.params)
    return study


In [ ]:
# --- Run CL study on fold f1 ---
# Note: adjust batch_size and n_batches as needed
study_cl_f1 = cl_run_f1(batch_size=3, n_batches=7, lie_policy="average")

# --- Load completed CL study ---
study = optuna.load_study(study_name="cl_f1", storage="sqlite:///cl_f1.db")
df_cl1 = study.trials_dataframe(attrs=("params", "user_attrs", "value", "state", "datetime_start", "datetime_complete"))
df_cl1.head(25)

In [ ]:
## --- Objective function for Fold 5 ---
# Note: same as Fold 1, but utility is averaged over two 5-year evaluation periods (2005–2009 + 2010–2014)

def submit_job_f5(trial, fold, label_prefix="cl_f5"):
    space = load_search_space()
    params = sample_params_from_space(trial, space, fold["req_start"], fold["req_end"])
    label = f"{label_prefix}_t{trial.number}"
    req_file = f"{label}.request"
    generate_request_from_dict(params, req_file)
    return label

def score_job_f5(label, fold, wait_hours=8, poll_secs=60):
    result_dir = RESULTS_BASE / label

    deadline = time.time() + wait_hours * 3600
    while time.time() < deadline:
        if result_dir.exists():
            break
        time.sleep(poll_secs)
    if not result_dir.exists():
        return float("-inf"), None, None, None, None, None

    zqq_path = make_zqq_csv(result_dir, label)

    # Compute utility as average over two evaluation periods (different to Fold 1)
    u1, f1a, f2a, f3a, f4a = utility_from_zqq(
        zqq_path,
        fold["eval1_start"], fold["eval1_end"], fold["crash1"],
        return_components=True
    )
    u2, f1b, f2b, f3b, f4b = utility_from_zqq(
        zqq_path,
        fold["eval2_start"], fold["eval2_end"], fold["crash2"],
        return_components=True
    )

    U  = (u1  + u2)  / 2.0
    f1 = (f1a + f1b) / 2.0
    f2 = (f2a + f2b) / 2.0
    f3 = (f3a + f3b) / 2.0
    f4 = (f4a + f4b) / 2.0

    return float(U), float(f1), float(f2), float(f3), float(f4), str(zqq_path)


def cl_run_f5(
    batch_size=4,
    n_batches=5,
    lie_policy="mean",         
    storage="sqlite:///cl_f5.db",
    study_name="cl_f5",
    seed=42
):
    fold = FOLDS["f5"] 

    study = optuna.create_study(direction="maximize", study_name=study_name,
                                storage=storage, load_if_exists=True,
                                sampler=optuna.samplers.TPESampler(seed=seed))

    shadow = optuna.create_study(direction=study.direction,
                                 sampler=optuna.samplers.TPESampler(seed=seed))

    def liar_value():
        vals = [t.value for t in study.trials if t.value is not None]
        if not vals:
            return 0.0
        if lie_policy == "best":
            return max(vals)
        if lie_policy == "mean":
            return mean(vals)
        if lie_policy == "pessimistic":
            return min(vals)
        return mean(vals)

    for b in range(n_batches):
        trials_real = []     
        labels = []          
        for _ in range(batch_size):
            t_real = study.ask()         
            t_shadow = shadow.ask()
            
            space = load_search_space()
            _ = sample_params_from_space(t_real,   space, fold["req_start"], fold["req_end"])
            _ = sample_params_from_space(t_shadow, space, fold["req_start"], fold["req_end"])

            shadow.tell(t_shadow, liar_value())

            label = submit_job_f5(t_real, fold, label_prefix=f"cl_f5_b{b}")
            trials_real.append(t_real)
            labels.append(label)

        for t_real, label in zip(trials_real, labels):

            U, f1, f2, f3, f4, zqq_path = score_job_f5(label, fold, wait_hours=8, poll_secs=60)
            t_real.set_user_attr("f1", f1)
            t_real.set_user_attr("f2", f2)
            t_real.set_user_attr("f3", f3)
            t_real.set_user_attr("f4", f4)
            t_real.set_user_attr("zqq_path", zqq_path)

            study.tell(t_real, float(U))
            print(f"[CL f5] trial {t_real.number} -> U={U:.4f} (f1={f1:.4f}, f2={f2:.4f}, f3={f3:.4f}, f4={f4:.4f})")

    print("CL f5 best value:", study.best_value)
    print("CL f5 best params:", study.best_trial.params)
    return study  


# --- Run CL study on fold f5 ---
# Note: adjust batch_size and n_batches as needed
study_cl_f5 = cl_run_f5(batch_size=4, n_batches=1, lie_policy="mean")


# --- Load completed CL study ---
study_cl5 = optuna.load_study(study_name="cl_f5", storage="sqlite:///cl_f5.db")
df_cl5 = study_cl5.trials_dataframe(attrs=("params", "user_attrs", "value", "state", "datetime_start", "datetime_complete"))
df_cl5.head(25)